# Convert Tensorflow model to ONNX
The general procedures to convert a tensorflow to ONNX is covered [part 1](./TensorflowToOnnx-1.ipynb).

In this tutorial, we will cover the following contents in order:
1. convert tensorflow model with other formats
   - convert with frozen graph
   - convert with checkpoint
2. convert in python script
3. useful command line options of `tensorflow-onnx`

## Convert with frozen graph
Tensorflow has API to get model's frozen graph and `tensorflow-onnx` can accept it as an input.

While besides the frozen graph, the input and output tensors' names are also needed. Those names typically end with ":0", you could either get them by tensorflow [summarized tool](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/tools/graph_transforms), or specify their names using tf.identity in your tensorflow script.

In [11]:
import tensorflow as tf
from assets.tensorflow_to_onnx_example import create_and_train_mnist

def save_model_to_frozen_proto(sess):
    frozen_graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, [output_tensor.name[:-2]])
    with open("./output/mnist_frozen.pb", "wb") as file:
         file.write(frozen_graph.SerializeToString())

sess_tf, saver, input_tensor, output_tensor = create_and_train_mnist()            
save_model_to_frozen_proto(sess_tf)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.08
step 1000, training accuracy 0.98
step 2000, training accuracy 1
step 3000, training accuracy 0.98
step 4000, training accuracy 1
test accuracy 0.984


In [12]:
# generating mnist.onnx using frozen_graph
!python -m tf2onnx.convert \
        --input ./output/mnist_frozen.pb \
        --inputs {input_tensor.name} \
        --outputs {output_tensor.name} \
        --output ./output/mnist2.onnx \
        --opset 7

2019-06-17 07:19:39,492 - INFO - Using tensorflow=1.12.0, onnx=1.5.0, tf2onnx=1.5.1/0c735a
2019-06-17 07:19:39,492 - INFO - Using opset <onnx, 7>
2019-06-17 07:19:39,606 - INFO - 
2019-06-17 07:19:39,635 - INFO - Optimizing ONNX model
2019-06-17 07:19:39,652 - INFO - After optimization: Add -2 (4->2), Identity -3 (3->0), Transpose -8 (9->1)
2019-06-17 07:19:39,654 - INFO - 
2019-06-17 07:19:39,654 - INFO - Successfully converted TensorFlow model ./output/mnist_frozen.pb to ONNX
2019-06-17 07:19:39,667 - INFO - ONNX model is saved at ./output/mnist2.onnx


## Convert with checkpoint
Same as frozen graph, you need to specify the path to checkpoint file and model's input and output names.

In [13]:
def save_model_to_checkpoint(saver, sess):
    save_path = saver.save(sess, "./output/ckpt/model.ckpt")

save_model_to_checkpoint(saver, sess_tf)

In [14]:
# generating mnist.onnx using checkpoint
!python -m tf2onnx.convert \
        --checkpoint ./output/ckpt/model.ckpt.meta \
        --inputs {input_tensor.name}\
        --outputs {output_tensor.name} \
        --output ./output/mnist3.onnx \
        --opset 7

2019-06-17 07:19:42,533 - INFO - Using tensorflow=1.12.0, onnx=1.5.0, tf2onnx=1.5.1/0c735a
2019-06-17 07:19:42,534 - INFO - Using opset <onnx, 7>
2019-06-17 07:19:42,660 - INFO - 
2019-06-17 07:19:42,684 - INFO - Optimizing ONNX model
2019-06-17 07:19:42,700 - INFO - After optimization: Add -2 (4->2), Identity -3 (3->0), Transpose -8 (9->1)
2019-06-17 07:19:42,705 - INFO - 
2019-06-17 07:19:42,705 - INFO - Successfully converted TensorFlow model ./output/ckpt/model.ckpt.meta to ONNX
2019-06-17 07:19:42,718 - INFO - ONNX model is saved at ./output/mnist3.onnx


## Convert in python script
`tensorflow-onnx` exports conversion APIs so that users can convert tensorflow model into ONNX directly in their script, the following code is an example.

In [15]:
from tf2onnx.tfonnx import process_tf_graph, tf_optimize
import tensorflow as tf
from tensorflow.graph_util import convert_variables_to_constants as freeze_graph

print("generating mnist.onnx in python script")
graph_def = freeze_graph(sess_tf, sess_tf.graph_def, [output_tensor.name[:-2]])
with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name='')
    onnx_graph = process_tf_graph(graph, opset=7, input_names=[input_tensor.name], output_names=[output_tensor.name])
model_proto = onnx_graph.make_model("test")
print("ONNX model is saved at ./output/mnist4.onnx")
with open("./output/mnist4.onnx", "wb") as f:
    f.write(model_proto.SerializeToString())

generating mnist.onnx in python script
ONNX model is saved at ./output/mnist4.onnx


## Useful command line options
The first useful option is "**opset**" which has been covered in [part 1](./TensorflowToOnnx-1.ipynb).

Then second option is "**inputs-as-nchw**". Tensorflow supports NCHW and NHWC while ONNX only supports NCHW for now, so if your model uses NHWC then the tool will insert extra transpose nodes to convert the model. And though "tensroflow-onnx" has optimizers to remove the transpose nodes as much as possible, it's suggested to use NCHW directly if possible. And if model with NCHW is impossible, this option will tell the tool that the real input format will be NCHW and it can remove more inserted transpose nodes now. For example --inputs input0:0,input1:0 --inputs-as-nchw input0:0 assumes that images are passed into input0:0 as nchw while the TensorFlow model given uses nhwc.

As said in part 1, ONNX defines its own operations set to represent machine learning computation operations and the set is different with tensorflow's. And two main difference will make the conversion fail, unsupported input dtype and unsupported operations, so `tensorflow-onnx` has two options to fix the gap if possible. The option "**target**" may insert cast operation to convert unsupported dtype into float in some target platform, please see the detail [here](https://github.com/onnx/tensorflow-onnx/wiki/target). The option "**custom-ops**" is useful when the runtime used supports custom ops that are not defined in onnx. For example: --custom-ops Print will insert a op Print in the onnx domain ai.onnx.converters.tensorflow into the graph.

More detail on `tensorflow-onnx` can be got from its [README](https://github.com/onnx/tensorflow-onnx/blob/master/README.md "Title") file,  for example the internal procedures in `tensorflow-onnx` to convert a tensorflow model.